In [1]:
# import libraries
import requests
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import string
import regex as re
import pygsheets
import os
import json
import gspread_dataframe as gd

In [2]:
# function that parse website and return dataframe
def parse():
    
    count = 0 # count variable
    df = pd.DataFrame({'Описание':[],'Адрес':[],'Цена':[], 'Ссылка':[]}) # empty dataframe

    while count < 300:
        r = requests.get(f'https://krisha.kz/prodazha/kvartiry/nur-sultan/?page={count}') # requsting connection with page
        html = BeautifulSoup(r.content, 'html.parser')
        a =html.findAll('div',class_="a-card__inc")


        for element in a:
            title = element.find('a',class_="a-card__title").get_text(strip = True)
            price = element.find('div',class_="a-card__price").get_text(strip = True)
            adress = element.find('div',class_="a-card__subtitle").get_text(strip = True)
            link = element.find('a', class_="a-card__title")['href']
            
            df = df.append({'Описание':title, # appending data to dataframe
                                         'Адрес':adress,
                                         'Цена':price,
                                           'Ссылка':link},
                                         ignore_index = True)
            
            

        count+=1
    return df

In [6]:
# function to clean dataframe
def cleanData(df):
    # before it was 44 000 000〒
    df['Цена'] = df['Цена'].str.replace('\D',' ')
    df['Цена'] = df['Цена'].str.replace(' ','')
    df['Цена'] = df['Цена'].astype('int64') 
    
    # remove letters from column, before it was, 3-комнатная квартира, 94 м², 10/11 этаж
    df['Описание'] = df['Описание'].str.replace('\D',' ')
    df['Количество комнат'] = df['Описание'].str[:1]
    df['Описание'] = df['Описание'].str[1:]
    
    # area of apartment
    squareArray = []
    for number in df['Описание']:
        squareArray.append(re.search('\d+|$', number).group()) # REGEX PATTERN to find first number

    df['Квадратный метр'] = squareArray
    df['Квадратный метр'] = df['Квадратный метр'].astype('int64')
    df.drop(['Описание'], axis = 'columns', inplace=True)
    
    # district of apartment
    splitAdress = []
    for el in df['Адрес']:
        splitAdress.append(el.split()[0]) # get first word 
    df['Район'] = splitAdress # new column in dataframe
    df = df.loc[(df['Район']=='Есильский') | (df['Район']=='Алматы') | (df['Район']=='Сарыарка')] # leave rows which contain specific words
    df = df.drop(['Адрес'], axis = 'columns') # drop column
    
    df['Цена за метр'] = (df['Цена']/df['Квадратный метр']).astype('int64')
    
    df['Ссылка'] = 'https://krisha.kz' + df['Ссылка']
    
    return df

In [7]:
# function to parse and clean page
def parseSheet():
    
    df = parse()
    df = cleanData(df)
    return df

In [8]:
import warnings
warnings.filterwarnings('ignore')

# assign dataframe
df = parseSheet()

In [9]:
# load json file
service_file = 'krishakz-2933fdb7b869.json'

# get connection to file
gc = pygsheets.authorize(service_file = service_file)

sheetName = 'KrishaKZ'
sh = gc.open(sheetName)

wks = sh.worksheet_by_title('test1')




In [10]:
wks.set_dataframe(df,(1,1), copy_index = True)

###  Link for google sheet
https://docs.google.com/spreadsheets/d/1WYTv5SD_Z7fMmUsjeiVnBYUu9jZPMXcXFRMkLP_rHTk/edit?usp=sharing

### Link for dashboard
https://datastudio.google.com/s/pGrcla4u8x4